# 결과 분석

## 목적
- 스코어_계산.ipynb에서 생성한 스코어 데이터를 분석
- 중증도 고위험군과 경제부담 고위험군 vs 산정특례질환 비교 분석
- 비산정특례질환 중 고위험군 확인 및 정책적 시사점 도출

## 전제조건
- 스코어_계산.ipynb가 먼저 실행되어 필요한 CSV 파일들이 생성되어 있어야 함

In [11]:
import os
os.chdir(r"c:\Users\user\OneDrive\바탕 화면\2025-2\DSL\EDA 프로젝트\dataset\전처리")
print(f"현재 작업 디렉토리: {os.getcwd()}")

현재 작업 디렉토리: c:\Users\user\OneDrive\바탕 화면\2025-2\DSL\EDA 프로젝트\dataset\전처리


In [12]:
import pandas as pd
import numpy as np

## 스코어 데이터 불러오기

In [13]:
# 스코어_계산.ipynb에서 생성된 통합 스코어 데이터 불러오기
df_result = pd.read_csv("스코어_통합.csv")
print(f"통합 스코어 데이터 로드 완료: {len(df_result)}개 질병")
print("데이터 컬럼:", list(df_result.columns))
print("\n데이터 미리보기:")
print(df_result.head())

통합 스코어 데이터 로드 완료: 286개 질병
데이터 컬럼: ['질병분류', '중증도_스코어', '경제부담_스코어']

데이터 미리보기:
   질병분류   중증도_스코어  경제부담_스코어
0     1  0.029546  0.640175
1    10  0.000000  0.705021
2   100  0.030881  0.401769
3   101  0.012678  0.587738
4   102  0.006671  0.672782


## 중증도 고위험군과 경제부담 고위험군 vs 산정특례질환 비교 분석

In [14]:
# 데이터 현황
print("데이터 현황:")
print(f"중증도/경제부담 스코어 데이터: {len(df_result)}개 질병")
print(f"전체 분석 대상: {len(df_result)}개 질병")

# 산정특례질환 리스트 불러오기
try:
    special = pd.read_csv('질병분류기준_산정특례여부추가.csv', encoding='utf-8-sig')
except:
    try:
        special = pd.read_csv('질병분류기준_산정특례여부추가.csv', encoding='cp949')
    except:
        special = pd.read_csv('질병분류기준_산정특례여부추가.csv', encoding='euc-kr')

print("산정특례질환 파일 컬럼:", list(special.columns))

# 산정특례질환 컬럼 지정
special_col = '산정특례여부'
print(f"사용할 산정특례질환 컬럼: {special_col}")

# 산정특례질환이 1 또는 'O'인 질병들 찾기
special_mask = (special[special_col] == 'O')
official_diseases = set(special[special_mask]['298 질병분류'].values)
print(f"산정특례질환 총 개수: {len(official_diseases)}개")

# 30% 기준 임계값 계산
total_diseases = len(df_result)
top30_count = int(total_diseases * 0.3)
print(f"\n상위 30% 기준: {top30_count}개 질병")

# 중증도 고위험군 분석
severity_threshold = df_result.nlargest(top30_count, '중증도_스코어')['중증도_스코어'].min()
severity_high_risk = df_result[df_result['중증도_스코어'] >= severity_threshold].copy()
severity_high_risk['official_flag'] = severity_high_risk['질병분류'].isin(official_diseases)

severity_official = sum(severity_high_risk['official_flag'])
severity_total = len(severity_high_risk)
severity_match_rate = (severity_official / severity_total) * 100

print(f"\n중증도 고위험군 vs 산정특례질환 일치도 분석")
print("="*60)
print(f"▶ 전체 질병 수                   : {total_diseases}")
print(f"▶ 상위 30% 개수                  : {top30_count}")
print(f"▶ 중증도 스코어 상위 30% 임계값   : {severity_threshold:.4f}")
print(f"▶ 중증도 고위험군 실제 개수       : {severity_total}")
print(f"▶ 중증도 고위험군 중 산정특례질환 : {severity_official}")
print(f"▶ 중증도 고위험군 일치도         : {severity_match_rate:.2f}%")

# 경제부담 고위험군 분석
burden_threshold = df_result.nlargest(top30_count, '경제부담_스코어')['경제부담_스코어'].min()
burden_high_risk = df_result[df_result['경제부담_스코어'] >= burden_threshold].copy()
burden_high_risk['official_flag'] = burden_high_risk['질병분류'].isin(official_diseases)

burden_official = sum(burden_high_risk['official_flag'])
burden_total = len(burden_high_risk)
burden_match_rate = (burden_official / burden_total) * 100

print(f"\n경제부담 고위험군 vs 산정특례질환 일치도 분석")
print("="*60)
print(f"▶ 전체 질병 수                   : {total_diseases}")
print(f"▶ 상위 30% 개수                  : {top30_count}")
print(f"▶ 경제부담 스코어 상위 30% 임계값 : {burden_threshold:.4f}")
print(f"▶ 경제부담 고위험군 실제 개수     : {burden_total}")
print(f"▶ 경제부담 고위험군 중 산정특례질환: {burden_official}")
print(f"▶ 경제부담 고위험군 일치도        : {burden_match_rate:.2f}%")

# 상위 20개 질병 상세 분석
print(f"\n중증도 고위험군 상위 20개 질병 상세:")
severity_top20 = severity_high_risk.nlargest(20, '중증도_스코어')[['질병분류', '중증도_스코어', 'official_flag']]
print(severity_top20.to_string(index=False))

print(f"\n경제부담 고위험군 상위 20개 질병 상세:")
burden_top20 = burden_high_risk.nlargest(20, '경제부담_스코어')[['질병분류', '경제부담_스코어', 'official_flag']]
print(burden_top20.to_string(index=False))

# 두 고위험군 겹침 분석
severity_diseases = set(severity_high_risk['질병분류'])
burden_diseases = set(burden_high_risk['질병분류'])
overlap_diseases = severity_diseases.intersection(burden_diseases)

print(f"\n중증도 고위험군과 경제부담 고위험군 겹침 분석:")
print(f"▶ 중증도 고위험군 질병 수      : {len(severity_diseases)}")
print(f"▶ 경제부담 고위험군 질병 수    : {len(burden_diseases)}")
print(f"▶ 두 고위험군 겹치는 질병 수   : {len(overlap_diseases)}")
print(f"▶ 겹침 비율 (중증도 기준)     : {len(overlap_diseases)/len(severity_diseases)*100:.2f}%")
print(f"▶ 겹침 비율 (경제부담 기준)   : {len(overlap_diseases)/len(burden_diseases)*100:.2f}%")

# 파일 저장
severity_high_risk.to_csv("중증도_고위험군.csv", index=False)
burden_high_risk.to_csv("경제부담_고위험군.csv", index=False)

print(f"\n완료: 각 고위험군 파일 저장됨")
print(f"  - '중증도_고위험군.csv' 저장됨 ({len(severity_high_risk)}개 질병)")
print(f"  - '경제부담_고위험군.csv' 저장됨 ({len(burden_high_risk)}개 질병)")

데이터 현황:
중증도/경제부담 스코어 데이터: 286개 질병
전체 분석 대상: 286개 질병
산정특례질환 파일 컬럼: ['298 질병분류', '298 질병분류_질병명', 'KCD 질병코드', '22대 질병분류', '22대 질병분류_질병명', '산정특례여부', '전부해당여부']
사용할 산정특례질환 컬럼: 산정특례여부
산정특례질환 총 개수: 90개

상위 30% 기준: 85개 질병

중증도 고위험군 vs 산정특례질환 일치도 분석
▶ 전체 질병 수                   : 286
▶ 상위 30% 개수                  : 85
▶ 중증도 스코어 상위 30% 임계값   : 0.0384
▶ 중증도 고위험군 실제 개수       : 85
▶ 중증도 고위험군 중 산정특례질환 : 50
▶ 중증도 고위험군 일치도         : 58.82%

경제부담 고위험군 vs 산정특례질환 일치도 분석
▶ 전체 질병 수                   : 286
▶ 상위 30% 개수                  : 85
▶ 경제부담 스코어 상위 30% 임계값 : 0.6274
▶ 경제부담 고위험군 실제 개수     : 85
▶ 경제부담 고위험군 중 산정특례질환: 14
▶ 경제부담 고위험군 일치도        : 16.47%

중증도 고위험군 상위 20개 질병 상세:
 질병분류  중증도_스코어  official_flag
   17 1.000000          False
   64 0.688965           True
   65 0.466769           True
   63 0.360766           True
   82 0.356076           True
   67 0.354616           True
   59 0.345667           True
  248 0.321663          False
  269 0.283316          False
  112 0.266597           True
   72 0.25

## 각 고위험군 중 비산정특례질환 확인

In [15]:
print("중증도 고위험군과 경제부담 고위험군 중 비산정특례질환 상세 분석")
print("=" * 70)

# 위의 셀에서 생성된 데이터 활용
# severity_high_risk: 중증도 고위험군
# burden_high_risk: 경제부담 고위험군
# special: 이미 불러온 산정특례질환 데이터 재사용

# 중복 컬럼 제거 (혹시 있을 경우를 대비)
disease_mapping = special.loc[:, ~special.columns.duplicated()]

# 질병분류와 질병명 매핑 딕셔너리 생성
disease_name_dict = dict(zip(
    disease_mapping['298 질병분류'].astype(int),
    disease_mapping['298 질병분류_질병명']
))

# 중증도 고위험군 중 비산정특례질환 추출 및 질병명 추가
severity_non_official = severity_high_risk[severity_high_risk['official_flag'] == False].copy()
severity_non_official['질병명'] = severity_non_official['질병분류'].map(disease_name_dict)

print(f"\n중증도 고위험군 중 비산정특례질환 분석:")
print(f"▶ 중증도 고위험군 총 개수: {len(severity_high_risk)}개")
print(f"▶ 중증도 고위험군 중 산정특례질환: {severity_high_risk['official_flag'].sum()}개")
print(f"▶ 중증도 고위험군 중 비산정특례질환: {len(severity_non_official)}개")
print(f"▶ 중증도 고위험군 중 비산정특례질환 비율: {len(severity_non_official)/len(severity_high_risk):.2%}")

# 경제부담 고위험군 중 비산정특례질환 추출 및 질병명 추가
burden_non_official = burden_high_risk[burden_high_risk['official_flag'] == False].copy()
burden_non_official['질병명'] = burden_non_official['질병분류'].map(disease_name_dict)

print(f"\n경제부담 고위험군 중 비산정특례질환 분석:")
print(f"▶ 경제부담 고위험군 총 개수: {len(burden_high_risk)}개")
print(f"▶ 경제부담 고위험군 중 산정특례질환: {burden_high_risk['official_flag'].sum()}개")
print(f"▶ 경제부담 고위험군 중 비산정특례질환: {len(burden_non_official)}개")
print(f"▶ 경제부담 고위험군 중 비산정특례질환 비율: {len(burden_non_official)/len(burden_high_risk):.2%}")

# 중증도 고위험 비산정특례질환 스코어 분석
print(f"\n중증도 고위험 비산정특례질환의 스코어 분석:")
print(f"▶ 평균 중증도 스코어: {severity_non_official['중증도_스코어'].mean():.4f}")
print(f"▶ 최대 중증도 스코어: {severity_non_official['중증도_스코어'].max():.4f}")
print(f"▶ 최소 중증도 스코어: {severity_non_official['중증도_스코어'].min():.4f}")

# 경제부담 고위험 비산정특례질환 스코어 분석
print(f"\n경제부담 고위험 비산정특례질환의 스코어 분석:")
print(f"▶ 평균 경제부담 스코어: {burden_non_official['경제부담_스코어'].mean():.4f}")
print(f"▶ 최대 경제부담 스코어: {burden_non_official['경제부담_스코어'].max():.4f}")
print(f"▶ 최소 경제부담 스코어: {burden_non_official['경제부담_스코어'].min():.4f}")

# 중증도 고위험군 비산정특례 상위 15개 질병 (질병명 포함)
if len(severity_non_official) > 0:
    print(f"\n중증도 고위험 비산정특례 질병 상위 15개:")
    top15_severity = severity_non_official.nlargest(15, '중증도_스코어')[
        ['질병분류', '질병명', '중증도_스코어', 'official_flag']
    ]
    print(top15_severity.to_string(index=False))

# 경제부담 고위험군 비산정특례 상위 15개 질병 (질병명 포함)
if len(burden_non_official) > 0:
    print(f"\n경제부담 고위험 비산정특례 질병 상위 15개:")
    top15_burden = burden_non_official.nlargest(15, '경제부담_스코어')[
        ['질병분류', '질병명', '경제부담_스코어', 'official_flag']
    ]
    print(top15_burden.to_string(index=False))

# 두 고위험군에 모두 포함되는 비산정특례질환 (핵심 분석)
severity_non_official_diseases = set(severity_non_official['질병분류'])
burden_non_official_diseases = set(burden_non_official['질병분류'])
overlap_non_official = severity_non_official_diseases & burden_non_official_diseases

print(f"\n중증도와 경제부담 모두 고위험인 비산정특례질환:")
print(f"▶ 중증도 고위험 비산정특례: {len(severity_non_official_diseases)}개")
print(f"▶ 경제부담 고위험 비산정특례: {len(burden_non_official_diseases)}개")
print(f"▶ 두 고위험군 모두 포함되는 비산정특례: {len(overlap_non_official)}개")

if len(overlap_non_official) > 0:
    # 겹치는 질병들의 상세 정보 생성
    overlap_severity = severity_non_official[severity_non_official['질병분류'].isin(overlap_non_official)][
        ['질병분류', '질병명', '중증도_스코어', 'official_flag']
    ]
    overlap_burden = burden_non_official[burden_non_official['질병분류'].isin(overlap_non_official)][
        ['질병분류', '경제부담_스코어']
    ]
    
    # 경제부담 스코어도 추가
    overlap_with_burden = pd.merge(
        overlap_severity, 
        overlap_burden, 
        on='질병분류'
    )
    
    print(f"\n중증도·경제부담 이중 고위험 비산정특례 질병 전체:")
    # 합산 스코어로 정렬
    overlap_with_burden['합산_스코어'] = overlap_with_burden['중증도_스코어'] + overlap_with_burden['경제부담_스코어']
    overlap_final = overlap_with_burden.sort_values('합산_스코어', ascending=False)[
        ['질병분류', '질병명', '중증도_스코어', '경제부담_스코어', '합산_스코어']
    ]
    print(overlap_final.to_string(index=False))
    
    # 이중 고위험 비산정특례질환 저장
    overlap_final.to_csv('이중고위험_비산정특례질환.csv', index=False, encoding='utf-8-sig')
    print(f"\n'이중고위험_비산정특례질환.csv' 저장됨 ({len(overlap_final)}개 질병)")

# 결과 저장
severity_non_official.to_csv('중증도_고위험_비산정특례질환.csv', index=False, encoding='utf-8-sig')
burden_non_official.to_csv('경제부담_고위험_비산정특례질환.csv', index=False, encoding='utf-8-sig')

print(f"\n분석 완료 및 파일 저장:")
print(f"  - '중증도_고위험_비산정특례질환.csv' 저장됨 ({len(severity_non_official)}개 질병)")
print(f"  - '경제부담_고위험_비산정특례질환.csv' 저장됨 ({len(burden_non_official)}개 질병)")
if len(overlap_non_official) > 0:
    print(f"  - '이중고위험_비산정특례질환.csv' 저장됨 ({len(overlap_non_official)}개 질병)")

print(f"\n정책적 시사점:")
print(f"  - 중증도와 경제부담 모두 고위험인 {len(overlap_non_official)}개 질병은 최우선 검토 대상")
print(f"  - 이들은 의학적으로도 심각하면서 경제적 부담도 큰 질병들")
print(f"  - 실손보험 정책 개선 시 가장 먼저 산정특례에 추가 고려할 질병들")

중증도 고위험군과 경제부담 고위험군 중 비산정특례질환 상세 분석

중증도 고위험군 중 비산정특례질환 분석:
▶ 중증도 고위험군 총 개수: 85개
▶ 중증도 고위험군 중 산정특례질환: 50개
▶ 중증도 고위험군 중 비산정특례질환: 35개
▶ 중증도 고위험군 중 비산정특례질환 비율: 41.18%

경제부담 고위험군 중 비산정특례질환 분석:
▶ 경제부담 고위험군 총 개수: 85개
▶ 경제부담 고위험군 중 산정특례질환: 14개
▶ 경제부담 고위험군 중 비산정특례질환: 71개
▶ 경제부담 고위험군 중 비산정특례질환 비율: 83.53%

중증도 고위험 비산정특례질환의 스코어 분석:
▶ 평균 중증도 스코어: 0.1361
▶ 최대 중증도 스코어: 1.0000
▶ 최소 중증도 스코어: 0.0384

경제부담 고위험 비산정특례질환의 스코어 분석:
▶ 평균 경제부담 스코어: 0.6833
▶ 최대 경제부담 스코어: 1.0000
▶ 최소 경제부담 스코어: 0.6274

중증도 고위험 비산정특례 질병 상위 15개:
 질병분류               질병명  중증도_스코어  official_flag
   17               폐혈증 1.000000          False
  248   자궁내 저산소증 및 출산질식 0.321663          False
  269                노쇠 0.283316          False
  149              폐색전증 0.226958          False
  128 뇌성마비 및 기타 마비성 증후군 0.211108          False
    7            호흡기 결핵 0.202210          False
  121              파킨슨병 0.193791          False
  122            알쯔하아머병 0.173532          False
   30           바이러스 뇌염 0.168767          False
  214          

## 분석 결과 요약

In [17]:
print("최종 분석 결과 요약")
print("=" * 50)

print(f"\n고위험군 분석 결과:")
print(f"▶ 중증도 고위험군 일치도: {severity_match_rate:.1f}%")
print(f"▶ 경제부담 고위험군 일치도: {burden_match_rate:.1f}%")

print(f"\n정책 사각지대:")
print(f"▶ 중증도 고위험 비산정특례: {len(severity_non_official)}개")
print(f"▶ 경제부담 고위험 비산정특례: {len(burden_non_official)}개")
print(f"▶ 이중 고위험 비산정특례: {len(overlap_non_official) if len(overlap_non_official) > 0 else 0}개")

print(f"\n정책 제언:")
print(f"▶ 최우선 검토 대상: 이중 고위험 비산정특례 {len(overlap_non_official) if len(overlap_non_official) > 0 else 0}개 질병")
print(f"▶ 중기 검토 대상: 단일 고위험 비산정특례질환들")
print(f"▶ 현재 정책의 개선 여지: 상당한 수준의 정책 사각지대 존재")

print(f"\n생성된 분석 파일:")
print(f"  - 중증도_고위험군.csv")
print(f"  - 경제부담_고위험군.csv")
print(f"  - 중증도_고위험_비산정특례질환.csv")
print(f"  - 경제부담_고위험_비산정특례질환.csv")
if len(overlap_non_official) > 0:
    print(f"  - 이중고위험_비산정특례질환.csv (최우선 검토 대상)")

최종 분석 결과 요약

고위험군 분석 결과:
▶ 중증도 고위험군 일치도: 58.8%
▶ 경제부담 고위험군 일치도: 16.5%

정책 사각지대:
▶ 중증도 고위험 비산정특례: 35개
▶ 경제부담 고위험 비산정특례: 71개
▶ 이중 고위험 비산정특례: 11개

정책 제언:
▶ 최우선 검토 대상: 이중 고위험 비산정특례 11개 질병
▶ 중기 검토 대상: 단일 고위험 비산정특례질환들
▶ 현재 정책의 개선 여지: 상당한 수준의 정책 사각지대 존재

생성된 분석 파일:
  - 중증도_고위험군.csv
  - 경제부담_고위험군.csv
  - 중증도_고위험_비산정특례질환.csv
  - 경제부담_고위험_비산정특례질환.csv
  - 이중고위험_비산정특례질환.csv (최우선 검토 대상)
